In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split
from torch.cuda.amp import GradScaler, autocast


In [23]:

def main():
    # Define transformations for the datasets
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load the dataset
    full_dataset = datasets.ImageFolder('/content/drive/MyDrive/JPEGImages', transform=transform)

    # Splitting the dataset into train and test
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    # Transform train dataset with additional augmentations
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        *transform.transforms
    ])
    train_dataset.dataset.transform = train_transform

    # Class weights calculation
    class_counts = torch.tensor([0.0] * 50)
    for _, label in full_dataset:
        class_counts[label] += 1
    class_weights = 1. / (class_counts + 1e-6)

    # Sample weights for the training dataset
    sample_weights = torch.tensor([class_weights[label] for _, label in [full_dataset.samples[i] for i in train_dataset.indices]])

    # Sampler for handling class imbalance
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

    # Data loaders
    dataloaders = {
        'train': DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=8, pin_memory=True),
        'test': DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)
    }

    # Setup the model
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_ftrs, 50),
        nn.BatchNorm1d(50),
        nn.ReLU()
    )

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scaler = GradScaler()

    # Use GPU if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Train the model
    total_start_time = time.time()
    for epoch in range(10):
        epoch_start_time = time.time()
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(dataloaders['train']):
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            if (i + 1) % 20 == 0:
                print(f'Epoch {epoch+1}, Step {i+1}, Loss: {running_loss / 20:.4f}')
                running_loss = 0.0

        epoch_end_time = time.time()
        print(f'Epoch {epoch+1} completed in {epoch_end_time - epoch_start_time:.2f} seconds')

    total_end_time = time.time()
    print(f'Total training time: {total_end_time - total_start_time:.2f} seconds')

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')





In [24]:
if __name__ == "__main__":
    main()


Epoch 1, Step 20, Loss: 2.9295
Epoch 1, Step 40, Loss: 2.3686
Epoch 1, Step 60, Loss: 2.1670
Epoch 1, Step 80, Loss: 2.1401
Epoch 1, Step 100, Loss: 1.9679
Epoch 1, Step 120, Loss: 1.8446
Epoch 1 completed in 32.80 seconds
Epoch 2, Step 20, Loss: 1.6954
Epoch 2, Step 40, Loss: 1.6367
Epoch 2, Step 60, Loss: 1.6431
Epoch 2, Step 80, Loss: 1.6104
Epoch 2, Step 100, Loss: 1.6350
Epoch 2, Step 120, Loss: 1.5211
Epoch 2 completed in 32.72 seconds
Epoch 3, Step 20, Loss: 1.5057
Epoch 3, Step 40, Loss: 1.4498
Epoch 3, Step 60, Loss: 1.3520
Epoch 3, Step 80, Loss: 1.4208
Epoch 3, Step 100, Loss: 1.3606
Epoch 3, Step 120, Loss: 1.2952
Epoch 3 completed in 33.00 seconds
Epoch 4, Step 20, Loss: 1.2890
Epoch 4, Step 40, Loss: 1.2367
Epoch 4, Step 60, Loss: 1.2493
Epoch 4, Step 80, Loss: 1.1506
Epoch 4, Step 100, Loss: 1.1405
Epoch 4, Step 120, Loss: 1.1722
Epoch 4 completed in 33.04 seconds
Epoch 5, Step 20, Loss: 1.1493
Epoch 5, Step 40, Loss: 1.1325
Epoch 5, Step 60, Loss: 1.0586
Epoch 5, Step 8